In [1]:
import os
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_path: Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [3]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_dir = os.path.dirname(config.checkpoint_model_path)

        create_directories(
            [
                Path(model_dir),
                Path(config.tensorboard_root_log_dir)
            ]
        )

        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_path=Path(config.checkpoint_model_path)
        )

        return prepare_callbacks_config

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "PetImages")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [5]:
import tensorflow as tf
import time

class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime('%Y-%m-%d-%H-%M-%S')
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f'tb_log_at_{timestamp}'
        )

        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_path,
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [6]:
import tensorflow as tf

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callbacks: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callbacks
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [7]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallbacks(config=prepare_callbacks_config)
    callbacks = prepare_callbacks.get_tb_ckpt_callbacks()
    
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callbacks=callbacks
    )
    
except Exception as e:
    raise e

[2022-09-18 17:32:18,679: INFO: common]: yaml file: configs\config.yaml loaded successfully
[2022-09-18 17:32:18,684: INFO: common]: yaml file: params.yaml loaded successfully
[2022-09-18 17:32:18,686: INFO: common]: created directory at: artifacts
[2022-09-18 17:32:18,687: INFO: common]: created directory at: artifacts\prepare_callbacks\checkpoint
[2022-09-18 17:32:18,689: INFO: common]: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir
[2022-09-18 17:32:18,691: INFO: common]: created directory at: artifacts\training
Found 4998 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
Epoch 1/20
 954/1250 [=====================>........] - ETA: 54s - loss: 6.9374 - accuracy: 0.6728

c:\Users\91955\miniconda3\envs\envdc38\lib\site-packages\PIL\TiffImagePlugin.py:845: UserWarning: Truncated File Read
  warnings.warn(str(msg))


1250/1250 [==============================] - 253s 198ms/step - loss: 6.1623 - accuracy: 0.6927 - val_loss: 3.6483 - val_accuracy: 0.7720
Epoch 2/20
1250/1250 [==============================] - 253s 202ms/step - loss: 3.2310 - accuracy: 0.7775 - val_loss: 0.9876 - val_accuracy: 0.9083
Epoch 3/20
1250/1250 [==============================] - 242s 193ms/step - loss: 2.5105 - accuracy: 0.8051 - val_loss: 1.1540 - val_accuracy: 0.8988
Epoch 4/20
1250/1250 [==============================] - 236s 189ms/step - loss: 2.4834 - accuracy: 0.8097 - val_loss: 1.3641 - val_accuracy: 0.8790
Epoch 5/20
1250/1250 [==============================] - 233s 187ms/step - loss: 2.2153 - accuracy: 0.8195 - val_loss: 0.7582 - val_accuracy: 0.9241
Epoch 6/20
1250/1250 [==============================] - 235s 188ms/step - loss: 2.1399 - accuracy: 0.8243 - val_loss: 2.1838 - val_accuracy: 0.8446
Epoch 7/20
1250/1250 [==============================] - 235s 188ms/step - loss: 1.9196 - accuracy: 0.8388 - val_loss: 0.754